<h2>3D stack - Single image - Marker+ based on colocalization</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) into a MIP and allows you to:

1. Inspect your images in Napari.
2. Define regions of interest (ROIs) using labels in Napari. Store said ROIs as .tiff files if needed.
3. Predict nuclei labels and store them as .tiff files for further processing.
4. Extract numbers of cells positive for a marker based on colocalization (using a user-defined min_max range).
5. Display positive cells in Napari.
6. Extract and save number of positive cells in a .csv file (marker_+_label_coloc_3D).

In [ ]:
from pathlib import Path
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from stardist.models import StarDist3D
import pyclesperanto_prototype as cle
from utils_stardist import get_gpu_details, list_images, read_image, extract_nuclei_stack, maximum_intensity_projection, save_rois, segment_marker_positive_nuclei, segment_nuclei_3d

get_gpu_details()
cle.select_device("RTX")

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [ ]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
markers = [("ki67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>. By changing the <code>slicing factor</code> you lose resolution but speed up processing times (check the results).

In [ ]:
# Explore each image to analyze (0 defines the first image in the directory)
image = images[0]

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3
n_tiles=(6,6,3)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
model_name = "MEC0.1"

# Model loading 
model = StarDist3D(None, name=model_name, basedir='stardist_models') 

# Generate maximum intensity projection and extract filename
img, filename = read_image (image, slicing_factor=None)

# Slice the nuclei stack
nuclei_img = extract_nuclei_stack(img, nuclei_channel)

# Generate maximum intensity projection 
img_mip = maximum_intensity_projection(img)

# Show image in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

<h3>Label your regions of interest in Napari and explore the signal of your marker of interest</h3>

Make sure to set <code>n edit dim = 3</code> so the label propagates across all channels. Name your regions of interest as i.e. <code>DG</code>, <code>CA1</code>, <code>CA3</code> or <code>HIPPO</code>. If you do not draw any ROI the entire image will be analyzed.

Fnally, select the <code>img_mip</code> layer and play with the contrast limit to later set a min_max range of intensities within which cells will be considered positive for said marker.

<video controls>
  <source src="../assets/napari_labels.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

<h3>Save user-defined label ROIs as .tiff files</h3>

In [ ]:
save_rois(viewer, directory_path, filename)

<h3>Define your analysis parameters</h3>

Modify the values for <code>nuclei_channel</code>, <code>marker_name</code>, <code>min_max_range</code> and <code>erosion_factor</code>

Define marker <code>location</code> as either "cytoplasm" or "nucleus", and the <code>cytoplasm_dil_radius</code>.

In [ ]:
# You can choose markers from the following list
markers

In [ ]:
# Type the marker you want to analyze from the list above
marker_name = 'neun'

# Define the intensity min_max range within which a cell is considered positive for a marker
# Useful for i.e. ignoring unspecific bright spots
min_max_range = (40, 100)

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
erosion_factor = 4

# Define the amount of pixels you want to add around the nuclei to simulate the cytoplasm
cytoplasm_dilation_radius = 2

# Retrieve the first and second values (channel and location) of the corresponding tuple in markers
for item in markers:
    if item[0] == marker_name:
        marker_channel = item[1]
        location = item[2]
        break  # Stop searching once the marker is found

# Slice marker stack
marker_img = img[marker_channel, :, :, :]

<h3>Mask the input image with the user defined labels and extract data</h3>

In [ ]:
# Add the 3D-stack into Napari
if segmentation_type == "3D":
    # Remove the 'img_mip' layer if it exists
    if 'img_mip' in viewer.layers:
        viewer.layers.remove('img_mip')
    # Add the 'img' stack
    viewer.add_image(img)

# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds" / segmentation_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

# Create an empty list to store all stats extracted from each image
stats = []

for roi_name in tqdm(roi_names):
    print(f"\nAnalyzing ROI: {roi_name}")

    # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
    try:
        # Read previously defined ROIs
        user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

    except FileNotFoundError:
        # Extract the xy dimensions of the input image 
        img_shape = img_mip.shape
        img_xy_dims = img_shape[-2:]

        # Create a label covering the entire image
        user_roi = np.ones(img_xy_dims).astype(np.uint8)

    # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
    try:
        # Read the nuclei predictions per ROI
        nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")
        print(f"Pre-computed nuclei labels found for {filename}")

    except FileNotFoundError:
        print(f"Generating nuclei labels for {filename}")

        # Slice the nuclei stack
        nuclei_img = extract_nuclei_stack(img, nuclei_channel)

        # We will create a mask where roi is greater than or equal to 1
        mask = (user_roi >= 1).astype(np.uint8)

        # 3D segmentation logic, extend 2D mask across the entire stack volume
        if segmentation_type == "3D":
            # Extract the number of z-slices to extend the mask
            slice_nr = img.shape[1]
            # Extend the mask across the entire volume
            mask = np.tile(mask, (slice_nr, 1, 1))
            # Apply the mask to nuclei_img, setting all other pixels to 0
            masked_nuclei_img = np.where(mask, nuclei_img, 0)
        else:
            # Apply the mask to nuclei_img, setting all other pixels to 0
            masked_nuclei_img = np.where(mask, nuclei_img, 0)

        # Segment nuclei and return labels
        nuclei_labels = segment_nuclei_3d(masked_nuclei_img, model, n_tiles)

        # Save nuclei labels as .tiff files to reuse them later
        try:
            os.makedirs(nuclei_preds_path / roi_name, exist_ok=True)
        except Exception as e:
            print(f"Error creating directory {nuclei_preds_path / roi_name}: {e}")

        # Construct path to store
        path_to_store = nuclei_preds_path / roi_name / f"{filename}.tiff"
        print(f"Saving nuclei labels to {path_to_store}")
        try:
            tifffile.imwrite(path_to_store, nuclei_labels)
        except Exception as e:
            print(f"Error saving file {path_to_store}: {e}")

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(nuclei_labels, name=f"{roi_name}_nuclei")

    # Add the ROIs as labels into Napari
    viewer.add_labels(user_roi, name=f"{roi_name}_ROI", opacity=0.4)

    # Simulate a cytoplasm by growing the nuclei_labels
    if location == "cytoplasm":
        print(f"Generating cytoplasm labels for: {marker_name}")
        nuclei_labels = cle.dilate_labels(nuclei_labels, radius=cytoplasm_dilation_radius)
        nuclei_labels = cle.pull(nuclei_labels)

    # Select marker positive nuclei
    nuclei_and_marker, eroded_nuclei_and_marker, marker_mip, processed_region_labels = segment_marker_positive_nuclei (nuclei_labels, marker_img, min_max_range, erosion_factor)
    viewer.add_image(nuclei_and_marker, name=f"{roi_name}_{marker_name}_nuclei_coloc")
    viewer.add_image(eroded_nuclei_and_marker, name=f"{roi_name}_{marker_name}_nuclei_eroded")
    viewer.add_labels(processed_region_labels, name=f"{roi_name}_{marker_name}+_nuclei")

    # Extract your information of interest
    total_nuclei = len(np.unique(nuclei_labels)) - 1
    marker_pos_nuclei = len(np.unique(processed_region_labels)) - 1

    # Calculate "%_marker+_cells" and avoid division by zero errors
    try:
        perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
    except ZeroDivisionError:
        perc_marker_pos_cells = 0

    # Create a dictionary containing all extracted info per masked image
    stats_dict = {
                "filename": filename,
                "ROI": roi_name,
                "marker": marker_name,
                "marker_location":location,
                "total_nuclei": total_nuclei,
                "marker+_nuclei": marker_pos_nuclei,
                "%_marker+_cells": perc_marker_pos_cells,
                "nuclei_ch": nuclei_channel,
                "marker_ch": marker_channel,
                "marker_min_max": min_max_range,
                "cytoplasm_dilation":cytoplasm_dilation_radius,
                "erosion_factor": erosion_factor,
                "slicing_factor": slicing_factor
                }

    # Append the current data point to the stats_list
    stats.append(stats_dict)    

<h3>Data saving</h3>


In [ ]:
# Define output folder for results
results_folder = Path("results") / experiment_id / segmentation_type / model_name

# Create the necessary folder structure if it does not exist
try:
    os.makedirs(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# Transform into a dataframe to store it as .csv later
df = pd.DataFrame(stats)

# Define the .csv path
csv_path = results_folder / f"SP_marker_+_label_coloc_{segmentation_type}.csv"

# Append to the .csv with new data points each round
df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df